<a href="https://colab.research.google.com/github/sourcecode369/TensorFlow-2.0/blob/master/TensorFlow-2.0/tensorflow_2.0_docs/TensorFlow%20Core/Guide/Tensorflow%202/Performance%20with%20tf.function/Notebook_1_Performance_with_tf_function_and_AutoGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install TensorFlow 2.0

In [38]:
!pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.0.0)


### Importing the dependencies

In [1]:
%%time
from __future__ import absolute_import, print_function, division, unicode_literals
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow import keras

CPU times: user 1.79 s, sys: 171 ms, total: 1.97 s
Wall time: 1.97 s


In [2]:
@tf.function
def simple_nn_layer(x, y):
    return tf.nn.relu(tf.matmul(x, y))

x = tf.random.uniform((3,3))
y = tf.random.uniform((3,3))

simple_nn_layer(x, y)

<tf.Tensor: id=23, shape=(3, 3), dtype=float32, numpy=
array([[0.40329936, 1.4176062 , 0.767733  ],
       [0.4518067 , 1.1736279 , 0.5299578 ],
       [0.09864775, 0.44770142, 0.23018213]], dtype=float32)>

In [3]:
simple_nn_layer

In [4]:
def linear_layer(x):
    return 2 * x + 1

@tf.function
def deep_net(x):
    return tf.nn.relu(linear_layer(x))

deep_net(tf.constant((1,2,3)))

<tf.Tensor: id=35, shape=(3,), dtype=int32, numpy=array([3, 5, 7], dtype=int32)>

In [5]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
    return conv_layer(image)

image = tf.zeros([1,200,200,100])
print("Eager conv: ", timeit.timeit(lambda: conv_layer(image), number=10))
print("Function conv: ", timeit.timeit(lambda: conv_fn(image), number=10))

Eager conv:  1.6871332350001467
Function conv:  1.7313378440003362


In [6]:
lstm_cell = tf.keras.layers.LSTMCell(10)

@tf.function
def lstm_fn(input, state):
    return lstm_cell(input, state)

input = tf.zeros([10, 10])
state = [tf.zeros([10,10])] * 2

lstm_cell(input, state)
print("Eager time: ", timeit.timeit(lambda: lstm_cell(input, state), number=10))
print("Function lstm: ", timeit.timeit(lambda: lstm_fn(input,state), number=10))

Eager time:  0.007938158000797557
Function lstm:  0.08429268700001558


### Using Python control flow

In [7]:
@tf.function
def square_if_positive(x):
    if x>0:
        x = x * x
    else:
        x = 0
    return x

print("square_if_positive(2) = {}".format(square_if_positive(tf.constant(2))))
print("square_if_positive(-2) = {}".format(square_if_positive(tf.constant(-2))))

square_if_positive(2) = 4
square_if_positive(-2) = 0


In [8]:
@tf.function
def sum_even(items):
    s = 0
    for c in items:
        if c % 2 > 0:
            continue
        s += c
    return s

sum_even(tf.constant([10, 12, 15, 20]))

<tf.Tensor: id=594, shape=(), dtype=int32, numpy=42>

In [9]:
print(tf.autograph.to_code(sum_even.python_function))

def tf__sum_even(items):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('sum_even', 'sum_even_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as sum_even_scope:
    s = 0

    def get_state_2():
      return ()

    def set_state_2(_):
      pass

    def loop_body(iterates, s):
      c = iterates
      continue_ = False

      def get_state():
        return ()

      def set_state(_):
        pass

      def if_true():
        continue_ = True
        return continue_

      def if_false():
        return continue_
      cond = c % 2 > 0
      continue_ = ag__.if_stmt(cond, if_true, if_false, get_state, set_state, ('continue_',), ())

      def get_state_1():
        return ()

      def set_state_1(_):
        pass

      def if_true_1():
        s_1, = s,
        s_1 += c
        return s_1

      def if_false_1():
        return s
      cond_1 = ag__.not_(continue_)
 

In [10]:
@tf.function
def fizzbuzz(x):
    for i in tf.range(1, x):
        if i%3 == 0 and i%5 == 0:
            tf.print("FizzBuzz")
        elif i%3 == 0:
            tf.print("Fizz")
        elif i%5 == 0:
            tf.print("Buzz", i)
        else:
            tf.print(i)

fizzbuzz(tf.constant(16))

1
2
Fizz
4
Buzz 5
Fizz
7
8
Fizz
Buzz 10
11
Fizz
13
14
FizzBuzz


### Keras and Autograph

In [11]:
class CustomModel(tf.keras.models.Model):
    @tf.function 
    def call(self, input_data):
        if tf.reduce_mean(input_data) > 0:
            return input_data 
        else:
            return input_data // 2

model = CustomModel()


print(model(tf.constant([-10,-20])))
print(model(tf.constant([10, 20])))

tf.Tensor([ -5 -10], shape=(2,), dtype=int32)
tf.Tensor([10 20], shape=(2,), dtype=int32)


### Side Effects

In [12]:
v = tf.Variable(5)

@tf.function
def find_next_odd():
    v.assign(v+1)
    if v%2 == 0:
        v.assign(v+1)
        tf.print(v)
find_next_odd()

7


In [13]:
print(tf.autograph.to_code(find_next_odd.python_function))

def tf__find_next_odd():
  with ag__.FunctionScope('find_next_odd', 'find_next_odd_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as find_next_odd_scope:
    ag__.converted_call(v.assign, find_next_odd_scope.callopts, (v + 1,), None, find_next_odd_scope)

    def get_state():
      return ()

    def set_state(_):
      pass

    def if_true():
      ag__.converted_call(v.assign, find_next_odd_scope.callopts, (v + 1,), None, find_next_odd_scope)
      ag__.converted_call(tf.print, find_next_odd_scope.callopts, (v,), None, find_next_odd_scope)
      return ag__.match_staging_level(1, cond)

    def if_false():
      return ag__.match_staging_level(1, cond)
    cond = v % 2 == 0
    ag__.if_stmt(cond, if_true, if_false, get_state, set_state, (), ())



### Debugging

In [0]:
@tf.function
def f(x):
    if x > 0:
        # import pdb
        # pdb.set_trace()
        x = x + 1
    return x

tf.config.experimental_run_functions_eagerly(True)

f(tf.constant(1))

tf.config.experimental_run_functions_eagerly(False)

### Advanced example: An in-graph training loop

#### Download Data

In [16]:
def prepare_mnist_features_and_labels(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y

def mnist_dataset():
    (x, y), _ = tf.keras.datasets.mnist.load_data()
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(prepare_mnist_features_and_labels)
    ds = ds.take(20000).shuffle(20000).batch(100)
    return ds

train_dataset = mnist_dataset()

11493376/11490434 [==============================] - 0s 0us/step


#### Define the model

In [0]:
model = tf.keras.Sequential([
                             tf.keras.layers.Reshape(target_shape=(28*28,), input_shape=(28,28)),
                             tf.keras.layers.Dense(512, activation = tf.nn.relu),
                             tf.keras.layers.Dense(100, activation=tf.nn.relu),
                             tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.build()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [27]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_4 (Reshape)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_9 (Dense)              (None, 100)               51300     
_________________________________________________________________
dense_10 (Dense)             (None, 10)                1010      
Total params: 454,230
Trainable params: 454,230
Non-trainable params: 0
_________________________________________________________________


#### Define the training loop

In [33]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()


def train_one_step(model, optimizer, x, y):
  with tf.GradientTape() as tape:
    logits = model(x)
    loss = compute_loss(y, logits)

  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  compute_accuracy(y, logits)
  return loss


@tf.function
def train(model, optimizer):
  train_ds = mnist_dataset()
  step = 0
  loss = 0.0
  accuracy = 0.0
  for x, y in train_ds:
    step += 1
    loss = train_one_step(model, optimizer, x, y)
    if step % 10 == 0:
      tf.print('Step', step, ': loss', loss, '; accuracy', compute_accuracy.result())
  return step, loss, accuracy

step, loss, accuracy = train(model, optimizer)
print('Final step', step, ': loss', loss, '; accuracy', compute_accuracy.result())

Step 10 : loss 1.50843143 ; accuracy 0.936
Step 20 : loss 1.50985682 ; accuracy 0.936
Step 30 : loss 1.55877042 ; accuracy 0.935
Step 40 : loss 1.53059983 ; accuracy 0.93675
Step 50 : loss 1.5172925 ; accuracy 0.935
Step 60 : loss 1.55291426 ; accuracy 0.936333358
Step 70 : loss 1.5751375 ; accuracy 0.9342857
Step 80 : loss 1.52663743 ; accuracy 0.933625
Step 90 : loss 1.52319646 ; accuracy 0.933666646
Step 100 : loss 1.54023981 ; accuracy 0.9356
Step 110 : loss 1.57837367 ; accuracy 0.935454547
Step 120 : loss 1.48585784 ; accuracy 0.937083304
Step 130 : loss 1.51173854 ; accuracy 0.937769234
Step 140 : loss 1.57104969 ; accuracy 0.937857151
Step 150 : loss 1.51294267 ; accuracy 0.938733339
Step 160 : loss 1.53978884 ; accuracy 0.938812494
Step 170 : loss 1.48750949 ; accuracy 0.938588262
Step 180 : loss 1.50173604 ; accuracy 0.938888907
Step 190 : loss 1.51377773 ; accuracy 0.939684212
Step 200 : loss 1.51598048 ; accuracy 0.94005
Final step tf.Tensor(200, shape=(), dtype=int32) : lo

### Batching

In [34]:
def square_if_positive(x):
    return [i**2 if i>0 else i for i in x]

square_if_positive(range(-10, 10))

[-10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [36]:
@tf.function
def square_if_positive(x):
    result = tf.TensorArray(tf.int32, size=x.shape[0])
    for i in tf.range(x.shape[0]):
        if x[i] > 0:
            result = result.write(i, x[i]**2)
        else:
            result = result.write(i, x[i])
    return result.stack()

square_if_positive(tf.range(-5,5))

<tf.Tensor: id=3180, shape=(10,), dtype=int32, numpy=array([-5, -4, -3, -2, -1,  0,  1,  4,  9, 16], dtype=int32)>

In [37]:
def square_if_positive(x):
    return tf.where(x>0, x**2, x)

square_if_positive(tf.range(-10,10))

<tf.Tensor: id=3189, shape=(20,), dtype=int32, numpy=
array([-10,  -9,  -8,  -7,  -6,  -5,  -4,  -3,  -2,  -1,   0,   1,   4,
         9,  16,  25,  36,  49,  64,  81], dtype=int32)>